# Evaluation on Turk Crowdsourced Aspect-Opinion Pair (3) Manual KP Multi-Aspect

In [1]:
import json
import pandas as pd
import numpy as np
# import spacy
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

In [2]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict

In [4]:
from utils.track_1_kp_matching import *

In [5]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
import torch

In [6]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader

First with top 10 args per category labelling of test set

## Load Test dataset

In [7]:
df = pd.read_pickle(f"../data/Evaluation/test_data.pkl")

In [8]:
df

,HITId,comment_id,label,full_key_point,key_point_id,comment,full_comment,attributes,topic,isMultiAspect,predicted_WA,num_of_token,old_key_point_id,key_point
0,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,1,Terrible administration and management .,kp_0_2,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,0.786076,8,kp_0_2,terrible administration <SEP> negative
1,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,1,Rude and Unprofessional staff .,kp_0_24,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,0.786076,8,kp_0_24,unprofessional staff <SEP> negative
2,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,0,Great for conventions and concerts ! !,kp_0_4,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,0.786076,8,kp_0_4,great for conventions <SEP> positive
3,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,0,Horrible customer service .,kp_0_5,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,0.786076,8,kp_0_5,horrible customer service <SEP> negative
4,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,0,It had wonderful acoustics .,kp_0_0,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,0.786076,8,kp_0_0,wonderful acoustics <SEP> positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6794,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Great family owned restaurant .,kp_4_25,[friendly staff <SEP> positive],The staff are friendly and attentive .,[staff],Restaurants,False,0.656356,6,kp_4_25,great family owned restaurant <SEP> positive
6795,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Worst service in the world !,kp_4_27,[friendly staff <SEP> positive],The staff are friendly and attentive .,[staff],Restaurants,False,0.656356,6,kp_4_27,worst service <SEP> negative
6796,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Rude and inattentive staff .,kp_4_30,[friendly staff <SEP> positive],The staff are friendly and attentive .,[staff],Restaurants,False,0.656356,6,kp_4_30,inattentive staff <SEP> negative
6797,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Great sushi and sashimi .,kp_4_31,[friendly staff <SEP> positive],The staff are friendly and attentive .,[staff],Restaurants,False,0.656356,6,kp_4_31,great sushi <SEP> positive


In [9]:
df['label'].value_counts()

0    6380
1     419
Name: label, dtype: int64

In [12]:
# df = df[df['topic'] == 'restaurant']

In [13]:
# df = df[df['topic'] == 'Hotels']
# df = df[df['topic'] == 'Restaurants']
# df = df[df['topic'] == 'Arts & Entertainment']
df = df[df['topic'] == 'Automotive']
# df = df[df['topic'] == 'Beauty & Spas']

In [14]:
# df = df[df['isMultiAspect'] == True]

### Multi-Aspect Configuration

In [15]:
df[df['arg_id'] == 'arg_1_6045']

,HITId,arg_id,label,full_key_point,key_point_id,argument,full_argument,attributes,topic,isMultiAspect,predicted_WA,num_of_token,old_key_point_id,key_point


In [16]:
len(df['full_argument'].unique())

48

In [17]:
df['num_of_aspects'] = df['argument'].str.len()

/tmp/ipykernel_1565/2261688680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num_of_aspects'] = df['argument'].str.len()


In [18]:
df = df.rename(columns={'arg_id': 'arg_id_sent'})
df['arg_id'] = df['arg_id_sent'] + "_" + df['num_of_aspects'].astype(str)
df

,HITId,arg_id_sent,label,full_key_point,key_point_id,argument,full_argument,attributes,topic,isMultiAspect,predicted_WA,num_of_token,old_key_point_id,key_point,num_of_aspects,arg_id
54,304QEQWKZPKY6KCW7FBWL3R7K3Y0OX,arg_1_1148,1,Over priced and overrated .,kp_1_6,"[terrible job <SEP> negative, zero quality <SEP> negative]","Terrible job , overpriced for zero quality .","[staff, restaurant -> atmosphere]",Automotive,True,0.728551,6,kp_1_6,over priced <SEP> negative,2,arg_1_1148_2
55,304QEQWKZPKY6KCW7FBWL3R7K3Y0OX,arg_1_1148,1,Overall just poor quality .,kp_1_8,"[terrible job <SEP> negative, zero quality <SEP> negative]","Terrible job , overpriced for zero quality .","[staff, restaurant -> atmosphere]",Automotive,True,0.728551,6,kp_1_8,poor quality <SEP> negative,2,arg_1_1148_2
56,304QEQWKZPKY6KCW7FBWL3R7K3Y0OX,arg_1_1148,1,TERRIBLE CUSTOMER SERVICE !,kp_1_1,"[terrible job <SEP> negative, zero quality <SEP> negative]","Terrible job , overpriced for zero quality .","[staff, restaurant -> atmosphere]",Automotive,True,0.728551,6,kp_1_1,terrible customer service <SEP> negative,2,arg_1_1148_2
57,304QEQWKZPKY6KCW7FBWL3R7K3Y0OX,arg_1_1148,0,Worst experience ever buying cars,kp_1_5,"[terrible job <SEP> negative, zero quality <SEP> negative]","Terrible job , overpriced for zero quality .","[staff, restaurant -> atmosphere]",Automotive,True,0.728551,6,kp_1_5,worst experience <SEP> negative,2,arg_1_1148_2
58,304QEQWKZPKY6KCW7FBWL3R7K3Y0OX,arg_1_1148,0,The cashiers are always friendly .,kp_1_0,"[terrible job <SEP> negative, zero quality <SEP> negative]","Terrible job , overpriced for zero quality .","[staff, restaurant -> atmosphere]",Automotive,True,0.728551,6,kp_1_0,always friendly cashiers <SEP> positive,2,arg_1_1148_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6726,3ZQA3IO31BRYD9XE3YFWZ8AHPV3O1N,arg_1_249,0,Very courteous staff .,kp_1_13,[rude everybody <SEP> negative],Everybody is so rude & disrespectful !,[staff],Automotive,False,0.670286,6,kp_1_13,very courteous staff <SEP> positive,1,arg_1_249_1
6727,3ZQA3IO31BRYD9XE3YFWZ8AHPV3O1N,arg_1_249,0,Excellent selection of vehicles,kp_1_14,[rude everybody <SEP> negative],Everybody is so rude & disrespectful !,[staff],Automotive,False,0.670286,6,kp_1_14,excellent selection of vehicles <SEP> positive,1,arg_1_249_1
6728,3ZQA3IO31BRYD9XE3YFWZ8AHPV3O1N,arg_1_249,0,"Free wifi , water , coffee , snacks !",kp_1_15,[rude everybody <SEP> negative],Everybody is so rude & disrespectful !,[staff],Automotive,False,0.670286,6,kp_1_15,free wifi water coffee <SEP> positive,1,arg_1_249_1
6729,3ZQA3IO31BRYD9XE3YFWZ8AHPV3O1N,arg_1_249,0,Their work is Impeccable .,kp_1_16,[rude everybody <SEP> negative],Everybody is so rude & disrespectful !,[staff],Automotive,False,0.670286,6,kp_1_16,impeccable work <SEP> positive,1,arg_1_249_1


In [19]:
df['num_of_aspects']

54      2
55      2
56      2
57      2
58      2
       ..
6726    1
6727    1
6728    1
6729    1
6730    1
Name: num_of_aspects, Length: 877, dtype: int64

In [20]:
# arg_df = df[['arg_id', 'topic', 'argument', 'full_argument']].drop_duplicates(subset=['arg_id']).reset_index(drop=True)
arg_df = df[['arg_id', 'topic', 'full_argument', 'num_of_aspects']].drop_duplicates(subset=['arg_id']).reset_index(drop=True)\
    .rename(columns={'full_argument': 'argument'})
arg_df

,arg_id,topic,argument,num_of_aspects
0,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2
1,arg_1_1558_1,Automotive,"They were professional , friendly and provided outstanding service .",1
2,arg_1_1153_1,Automotive,"Terry Repo is a consummate salesman , very honest and approachable .",1
3,arg_1_392_1,Automotive,Great job failing miserably with a valuable client .,1
4,arg_1_1430_1,Automotive,The waiting area was clean and comfortable .,1
5,arg_1_1260_1,Automotive,The entire team was patient and informative .,1
6,arg_1_823_1,Automotive,Jim Marsh agents worst service to clients !,1
7,arg_1_1520_2,Automotive,They have great offers and awesome customer service too .,2
8,arg_1_1402_1,Automotive,The staff is friendly and courteous .,1
9,arg_1_1550_1,Automotive,They treated me with respect and were friendly and helpful .,1


In [21]:
# kp_df = df[['key_point_id', 'topic', 'key_point', 'full_key_point']].drop_duplicates(subset=['key_point_id']).reset_index(drop=True)
kp_df = df[['key_point_id', 'topic', 'full_key_point']].drop_duplicates(subset=['key_point_id']).reset_index(drop=True)\
    .rename(columns={'full_key_point': 'key_point'})
kp_df

,key_point_id,topic,key_point
0,kp_1_6,Automotive,Over priced and overrated .
1,kp_1_8,Automotive,Overall just poor quality .
2,kp_1_1,Automotive,TERRIBLE CUSTOMER SERVICE !
3,kp_1_5,Automotive,Worst experience ever buying cars
4,kp_1_0,Automotive,The cashiers are always friendly .
5,kp_1_2,Automotive,They have excellent customer service .
6,kp_1_3,Automotive,"employees looked overworked , confused ."
7,kp_1_4,Automotive,The employees here are wonderful !
8,kp_1_7,Automotive,Prices were fair and reasonable .
9,kp_1_9,Automotive,FAST and FRIENDLY SERVICE ! ! !


In [22]:
labels_df = df[['arg_id', 'key_point_id', 'label']]
# labels_df = df[['arg_id', 'key_point_id', 'weakly_supervised_label']].rename(columns={'weakly_supervised_label': 'label'})
labels_df

,arg_id,key_point_id,label
54,arg_1_1148_2,kp_1_6,1
55,arg_1_1148_2,kp_1_8,1
56,arg_1_1148_2,kp_1_1,1
57,arg_1_1148_2,kp_1_5,0
58,arg_1_1148_2,kp_1_0,0
...,...,...,...
6726,arg_1_249_1,kp_1_13,0
6727,arg_1_249_1,kp_1_14,0
6728,arg_1_249_1,kp_1_15,0
6729,arg_1_249_1,kp_1_16,0


## Opinion Retrieval

In [23]:
root_data_path = "../../"

In [24]:
# eval_yelp_df = pd.read_pickle(root_data_path + "./data/snippext_rest_hotel_eval_sample_dataset_save/eval_yelp_df.pkl")
eval_yelp_df = pd.read_pickle(root_data_path + "./data/snippext_top_5_domain_eval_sample_dataset_save/eval_yelp_df.pkl")
eval_yelp_df

domain  \
index                    
260808  0       Hotels   
        1       Hotels   
        2       Hotels   
260809  0       Hotels   
        1       Hotels   
...                ...   
1035123 4  Restaurants   
1035124 0  Restaurants   
        1  Restaurants   
        2  Restaurants   
        3  Restaurants   

                                                                                                  categories_list  \
index                                                                                                               
260808  0                             [Hotels & Travel, Venues & Event Spaces, Event Planning & Services, Hotels]   
        1                             [Hotels & Travel, Venues & Event Spaces, Event Planning & Services, Hotels]   
        2                             [Hotels & Travel, Venues & Event Spaces, Event Planning & Services, Hotels]   
260809  0                             [Hotels & Travel, Venues & Event Spaces, Event Planning & Services, Hotels]   
        1                             [Hotels & Travel, Venues & Event Spaces, Event Planning & Services, Hotels]   
...                                                                                                           ...   
1035123 4  [American (Traditional), Sandwiches, Pizza, Brewpubs, Burgers, Restaurants, Breweries, Food, Desserts]   
1035124 0  [American (Traditional), Sandwiches, Pizza, Brewpubs, Burgers, Restaurants, Breweries, Food, Desserts]   
        1  [American (Traditional), Sandwiches, Pizza, Brewpubs, Burgers, Restaurants, Breweries, Food, Desserts]   
        2  [American (Traditional), Sandwiches, Pizza, Brewpubs, Burgers, Restaurants, Breweries, Food, Desserts]   
        3  [American (Traditional), Sandwiches, Pizza, Brewpubs, Burgers, Restaurants, Breweries, Food, Desserts]   

                                                                                                     categories  \
index                                                                                                             
260808  0                             Hotels & Travel, Venues & Event Spaces, Event Planning & Services, Hotels   
        1                             Hotels & Travel, Venues & Event Spaces, Event Planning & Services, Hotels   
        2                             Hotels & Travel, Venues & Event Spaces, Event Planning & Services, Hotels   
260809  0                             Hotels & Travel, Venues & Event Spaces, Event Planning & Services, Hotels   
        1                             Hotels & Travel, Venues & Event Spaces, Event Planning & Services, Hotels   
...                                                                                                         ...   
1035123 4  American (Traditional), Sandwiches, Pizza, Brewpubs, Burgers, Restaurants, Breweries, Food, Desserts   
1035124 0  American (Traditional), Sandwiches, Pizza, Brewpubs, Burgers, Restaurants, Breweries, Food, Desserts   
        1  American (Traditional), Sandwiches, Pizza, Brewpubs, Burgers, Restaurants, Breweries, Food, Desserts   
        2  American (Traditional), Sandwiches, Pizza, Brewpubs, Burgers, Restaurants, Breweries, Food, Desserts   
        3  American (Traditional), Sandwiches, Pizza, Brewpubs, Burgers, Restaurants, Breweries, Food, Desserts   

                                             name             business_id  \
index                                                                       
260808  0  Courtyard by Marriott Toronto Downtown  XHrPQKUTJXJGPH9etKUYUw   
        1  Courtyard by Marriott Toronto Downtown  XHrPQKUTJXJGPH9etKUYUw   
        2  Courtyard by Marriott Toronto Downtown  XHrPQKUTJXJGPH9etKUYUw   
260809  0  Courtyard by Marriott Toronto Downtown  XHrPQKUTJXJGPH9etKUYUw   
        1  Courtyard by Marriott Toronto Downtown  XHrPQKUTJXJGPH9etKUYUw   
...                                           ...                     ...   
1035123 4                        The Grumpy

In [25]:
sentences = arg_df['argument'].tolist()
opinion_argument_df = eval_yelp_df[eval_yelp_df['sentences'].isin(sentences)][['sentences', 'aspects', 'opinions', 'opinion_aspect_pairs', 'sentiments', 'attributes']]
opinion_argument_df = opinion_argument_df.rename(columns={'sentences': 'argument'})
arg_df = arg_df.merge(opinion_argument_df, on=['argument'])

In [26]:
yelp_kp_df = pd.read_csv("../../../ABSA Dataset/Manual Label/snippext_top_5_domain_eval_data_arg_quality_key_points_test_set_annotated.csv")
yelp_kp_df = yelp_kp_df[yelp_kp_df['isSelect'] == True].reset_index(drop=True).rename(columns={'domain': 'topic'})

In [27]:
yelp_kp_df

,index,level_1,id,business_id,attributes,topic,opinion_aspect_pairs,opinions,aspects,sentiments,key_point,predicted_WA,isSelect
0,773765,3,773765_3,MHq5Ly9ZMbCgnP3PoQJbVQ,restaurant -> atmosphere,Arts & Entertainment,['wonderful acoustics'],wonderful,acoustics,['positive'],It had wonderful acoustics .,0.574064,True
1,720169,2,720169_2,1wiv2BvvauMj8hS16UuEYw,restaurant -> atmosphere,Arts & Entertainment,['poor acoustics'],poor,acoustics,['negative'],The acoustics are poor .,0.452797,True
2,773741,1,773741_1,MHq5Ly9ZMbCgnP3PoQJbVQ,staff,Arts & Entertainment,['terrible administration'],terrible,administration,['negative'],Terrible administration and management .,0.434350,True
3,741407,4,741407_4,UYZ_LzFRuWTv__VsXUXhsg,restaurant -> atmosphere,Arts & Entertainment,['filthy bathrooms'],filthy,bathrooms,['negative'],Bathrooms were filthy .,0.479426,True
4,816968,3,816968_3,rMrymOj6RcBBddGuOjv_IQ,food -> variety,Arts & Entertainment,['great for conventions'],great for,conventions,['positive'],Great for conventions and concerts ! !,0.457361,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,26080900,1,26080900_1,XHrPQKUTJXJGPH9etKUYUw,value-for-money,Arts & Entertainment,['overvalued price'],overvalued,price,['negative'],Price way overvalued .,0.494802,True
136,26080901,1,26080901_1,XHrPQKUTJXJGPH9etKUYUw,value-for-money,Beauty & Spas,['overvalued price'],overvalued,price,['negative'],Price way overvalued .,0.494802,True
137,78277800,2,78277800_2,UyAN1BLsH5rPJjMvdYKtHg,staff,Hotels,['extremely poor service'],extremely poor,service,['negative'],Extremely poor service .,0.557798,True
138,103402600,4,103402600_4,8Zq11pzOup3-iwOjdMMwDQ,restaurant -> atmosphere,Arts & Entertainment,['clean everything'],clean,everything,['positive'],Everything was clean and welcoming .,0.460879,True


In [28]:
kp_df = kp_df.merge(yelp_kp_df[['key_point', 'aspects', 'opinions', 'opinion_aspect_pairs', 'sentiments', 'topic', 'attributes', 'predicted_WA']], 
                    on=['key_point', 'topic'])

In [29]:
kp_df

,key_point_id,topic,key_point,aspects,opinions,opinion_aspect_pairs,sentiments,attributes,predicted_WA
0,kp_1_6,Automotive,Over priced and overrated .,priced,over,['over priced'],['negative'],value-for-money,0.550575
1,kp_1_8,Automotive,Overall just poor quality .,quality,poor,['poor quality'],['negative'],restaurant -> atmosphere,0.530240
2,kp_1_1,Automotive,TERRIBLE CUSTOMER SERVICE !,customer service,terrible,['terrible customer service'],['negative'],staff,0.626694
3,kp_1_5,Automotive,Worst experience ever buying cars,experience,worst,['worst experience'],['negative'],restaurant -> atmosphere,0.471267
4,kp_1_0,Automotive,The cashiers are always friendly .,cashiers,always friendly,['always friendly cashiers'],['positive'],staff,0.429570
5,kp_1_2,Automotive,They have excellent customer service .,customer service,excellent,['excellent customer service'],['positive'],staff,0.585227
6,kp_1_3,Automotive,"employees looked overworked , confused .",employees,overworked,['overworked employees'],['negative'],staff,0.506310
7,kp_1_4,Automotive,The employees here are wonderful !,employees,wonderful,['wonderful employees'],['positive'],staff,0.443008
8,kp_1_7,Automotive,Prices were fair and reasonable .,prices,fair and reasonable,['fair and reasonable prices'],['positive'],value-for-money,0.499178
9,kp_1_9,Automotive,FAST and FRIENDLY SERVICE ! ! !,service,fast and friendly,['fast and friendly service'],['positive'],wait-time,0.561472


## Merging

In [30]:
arg_df[arg_df['num_of_aspects'] > 1]

,arg_id,topic,argument,num_of_aspects,aspects,opinions,opinion_aspect_pairs,sentiments,attributes
0,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,"[job, quality]","[terrible, zero]","[terrible job, zero quality]","[negative, negative]","[staff, restaurant -> atmosphere]"
8,arg_1_1520_2,Automotive,They have great offers and awesome customer service too .,2,"[offers, customer service]","[great, awesome]","[great offers, awesome customer service]","[positive, positive]","[food -> variety, staff]"
14,arg_1_28_2,Automotive,"Adam , manager of the store was welcoming and reassuring .",2,"[store, manager]","[welcoming, reassuring]","[welcoming store, reassuring manager]","[positive, positive]","[staff, staff]"
18,arg_1_431_2,Automotive,"Great service from a young , energetic team from start to finish .",2,"[service, team]","[great, energetic]","[great service, energetic team]","[positive, positive]","[staff, restaurant -> atmosphere]"
23,arg_1_1454_2,Automotive,Their dedication to resolving this major safety problem was incredible .,2,"[safety problem, dedication]","[incredible, major]","[incredible safety problem, major dedication]","[positive, positive]","[restaurant -> atmosphere, restaurant -> atmosphere]"
24,arg_1_176_2,Automotive,Cheap car wash with AMAZING employees !,2,"[car wash, employees]","[cheap, amazing]","[cheap car wash, amazing employees]","[positive, positive]","[food -> quality, staff]"
33,arg_1_503_2,Automotive,"Honest service , honest sales , and top quality craftsmanship !",2,"[service, craftsmanship]","[honest, top quality]","[honest service, top quality craftsmanship]","[positive, positive]","[staff, restaurant -> atmosphere]"
37,arg_1_321_2,Automotive,Friendly staff and the services offered are great .,2,"[staff, services offered]","[friendly, great]","[friendly staff, great services offered]","[positive, positive]","[staff, staff]"


In [31]:
arg_df = arg_df.explode(['aspects', 'opinions', 'opinion_aspect_pairs', 'sentiments', 'attributes'])

In [32]:
arg_df

,arg_id,topic,argument,num_of_aspects,aspects,opinions,opinion_aspect_pairs,sentiments,attributes
0,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff
0,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,quality,zero,zero quality,negative,restaurant -> atmosphere
1,arg_1_1558_1,Automotive,"They were professional , friendly and provided outstanding service .",1,service,outstanding,outstanding service,positive,staff
2,arg_1_1153_1,Automotive,"Terry Repo is a consummate salesman , very honest and approachable .",1,salesman,approachable,approachable salesman,positive,staff
3,arg_1_1153_1,Automotive,"Terry Repo is a consummate salesman , very honest and approachable .",1,salesman,approachable,approachable salesman,positive,staff
4,arg_1_392_1,Automotive,Great job failing miserably with a valuable client .,1,job,great,great job,positive,food -> quality
5,arg_1_1430_1,Automotive,The waiting area was clean and comfortable .,1,waiting area,clean,clean waiting area,positive,wait-time
6,arg_1_1260_1,Automotive,The entire team was patient and informative .,1,team,patient,patient team,positive,staff
7,arg_1_823_1,Automotive,Jim Marsh agents worst service to clients !,1,service,worst,worst service,negative,staff
8,arg_1_1520_2,Automotive,They have great offers and awesome customer service too .,2,offers,great,great offers,positive,food -> variety


In [33]:
import ast
kp_df['opinion_aspect_pairs'] = kp_df['opinion_aspect_pairs'].apply(lambda x: ast.literal_eval(x))
kp_df['sentiments'] = kp_df['sentiments'].apply(lambda x: ast.literal_eval(x))
kp_df = kp_df.explode(['opinion_aspect_pairs', 'sentiments'])
kp_df

,key_point_id,topic,key_point,aspects,opinions,opinion_aspect_pairs,sentiments,attributes,predicted_WA
0,kp_1_6,Automotive,Over priced and overrated .,priced,over,over priced,negative,value-for-money,0.550575
1,kp_1_8,Automotive,Overall just poor quality .,quality,poor,poor quality,negative,restaurant -> atmosphere,0.530240
2,kp_1_1,Automotive,TERRIBLE CUSTOMER SERVICE !,customer service,terrible,terrible customer service,negative,staff,0.626694
3,kp_1_5,Automotive,Worst experience ever buying cars,experience,worst,worst experience,negative,restaurant -> atmosphere,0.471267
4,kp_1_0,Automotive,The cashiers are always friendly .,cashiers,always friendly,always friendly cashiers,positive,staff,0.429570
5,kp_1_2,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227
6,kp_1_3,Automotive,"employees looked overworked , confused .",employees,overworked,overworked employees,negative,staff,0.506310
7,kp_1_4,Automotive,The employees here are wonderful !,employees,wonderful,wonderful employees,positive,staff,0.443008
8,kp_1_7,Automotive,Prices were fair and reasonable .,prices,fair and reasonable,fair and reasonable prices,positive,value-for-money,0.499178
9,kp_1_9,Automotive,FAST and FRIENDLY SERVICE ! ! !,service,fast and friendly,fast and friendly service,positive,wait-time,0.561472


In [34]:
# Should not include attribute during matching to not favouring our apporach during evaluation, because other baselines don't
# use attribute as well during the matching
sent_kp_df = arg_df.astype(str).merge(kp_df.astype(str), how='left', on=['topic']).dropna(subset=['key_point']) 
sent_kp_df = sent_kp_df.reset_index(drop=True)
sent_kp_df

,arg_id,topic,argument,num_of_aspects,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA
0,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_6,Over priced and overrated .,priced,over,over priced,negative,value-for-money,0.550575316
1,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_8,Overall just poor quality .,quality,poor,poor quality,negative,restaurant -> atmosphere,0.530240297
2,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_1,TERRIBLE CUSTOMER SERVICE !,customer service,terrible,terrible customer service,negative,staff,0.626694322
3,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_5,Worst experience ever buying cars,experience,worst,worst experience,negative,restaurant -> atmosphere,0.471266836
4,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_0,The cashiers are always friendly .,cashiers,always friendly,always friendly cashiers,positive,staff,0.429570019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_13,Very courteous staff .,staff,very courteous,very courteous staff,positive,staff,0.427563697
1022,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_14,Excellent selection of vehicles,vehicles,excellent selection of,excellent selection of vehicles,positive,food -> variety,0.458006501
1023,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_15,"Free wifi , water , coffee , snacks !",wifi water coffee,free,free wifi water coffee,positive,wait-time,0.539548993
1024,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_16,Their work is Impeccable .,work,impeccable,impeccable work,positive,staff,0.446787089


In [35]:
# sent_kp_df = arg_df.astype(str).merge(kp_df.astype(str), how='left', on=['topic', 'attributes']).dropna(subset=['key_point'])
# sent_kp_df = sent_kp_df.reset_index(drop=True)
# sent_kp_df

In [36]:
# sent_kp_df.drop_duplicates(subset=['opinion_aspect_pairs_x', 'opinion_aspect_pairs_y'])

In [37]:
# sent_kp_df[sent_kp_df['opinion_aspect_pairs_x'] == sent_kp_df['opinion_aspect_pairs_y']]

In [38]:
sent_kp_df.loc[
    (sent_kp_df['aspects_x'] == sent_kp_df['aspects_y']) & (sent_kp_df['sentiments_x'] == sent_kp_df['sentiments_y']),
    'score'
] = 1

In [39]:
sent_kp_df.loc[
    (sent_kp_df['sentiments_x'] != sent_kp_df['sentiments_y']),
    'score'
] = 0

In [40]:
sent_kp_df[sent_kp_df['score'] == 0]

,arg_id,topic,argument,num_of_aspects,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA,score
4,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_0,The cashiers are always friendly .,cashiers,always friendly,always friendly cashiers,positive,staff,0.429570019,0.0
5,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_2,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227013,0.0
7,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_4,The employees here are wonderful !,employees,wonderful,wonderful employees,positive,staff,0.443008006,0.0
8,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_7,Prices were fair and reasonable .,prices,fair and reasonable,fair and reasonable prices,positive,value-for-money,0.499178082,0.0
9,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_9,FAST and FRIENDLY SERVICE ! ! !,service,fast and friendly,fast and friendly service,positive,wait-time,0.561472356,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_13,Very courteous staff .,staff,very courteous,very courteous staff,positive,staff,0.427563697,0.0
1022,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_14,Excellent selection of vehicles,vehicles,excellent selection of,excellent selection of vehicles,positive,food -> variety,0.458006501,0.0
1023,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_15,"Free wifi , water , coffee , snacks !",wifi water coffee,free,free wifi water coffee,positive,wait-time,0.539548993,0.0
1024,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_16,Their work is Impeccable .,work,impeccable,impeccable work,positive,staff,0.446787089,0.0


Handle implicit match

In [41]:
# import spacy

# nlp = spacy.load('en_core_web_md')

# print("Enter two space-separated words")
# words = input()

# tokens = nlp(words)

# for token in tokens:
#     # Printing the following attributes of each token.
#     # text: the word string, has_vector: if it contains
#     # a vector representation in the model,
#     # vector_norm: the algebraic norm of the vector,
#     # is_oov: if the word is out of vocabulary.
#     print(token.text, token.has_vector, token.vector_norm, token.is_oov)

# token1, token2 = tokens[0], tokens[1:]

# print("Similarity:", token1.similarity(token2))

In [42]:
import spacy

nlp = spacy.load('en_core_web_md')

2023-06-19 04:49:02.929407: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-19 04:49:03.871614: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-19 04:49:03.872067: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-19 04:49:03.872233: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.77GHz coreCount: 80 deviceMemorySize: 12.00GiB deviceMemoryBandwidth: 849.46GiB/s
2023-06-19 04:49:03.872283: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-19 04:49:03.872316: I tensorflow/stream_ex

In [43]:
# import spacy

# nlp = spacy.load('en_core_web_md')

# print("Enter two space-separated words")
# words = input()

# tokens = nlp(words)

# for token in tokens:
#     # Printing the following attributes of each token.
#     # text: the word string, has_vector: if it contains
#     # a vector representation in the model,
#     # vector_norm: the algebraic norm of the vector,
#     # is_oov: if the word is out of vocabulary.
#     print(token.text, token.has_vector, token.vector_norm, token.is_oov)

# sep_index = [token.i for token in tokens if token.text == '|'][0]
    
# # token1, token2 = tokens[0], tokens[1:]

# # print("Similarity:", token1.similarity(token2))

In [44]:
# token1, token2 = tokens[:sep_index], tokens[sep_index+1:]
# token1.similarity(token2)

In [45]:
# token2.similarity(token1)

In [46]:
import warnings
warnings.filterwarnings("error")

In [47]:
sent_kp_df['predicted_by_cosine'] = False

In [48]:
unfound_tokens = []
def calculate_aspects_semantic_similarity(row):
    tokens = nlp(row['aspects_x'] + " | " + row['aspects_y'])
    sep_index = [token.i for token in tokens if token.text == '|'][0]
    token1, token2 = tokens[:sep_index], tokens[sep_index+1:]
    row['aspects_x_len'] = len(token1)
    row['aspects_y_len'] = len(token2)
#     print(token1, token2)
    try:
        print(token2.similarity(token1))
        row['score'] = token2.similarity(token1)
    except:
#         display(row)
#         unfound_tokens += [token1]
        row['score'] = 0
    row['predicted_by_cosine'] = True
    
    return row

In [49]:
# implicit_match_analyze_mask = sent_kp_df['aspects_x'] != sent_kp_df['aspects_y']
implicit_match_analyze_mask = pd.isnull(sent_kp_df['score'])
sent_kp_df[implicit_match_analyze_mask] = sent_kp_df[implicit_match_analyze_mask].apply(calculate_aspects_semantic_similarity, axis=1)

0.09381703287363052
0.21549391746520996
0.3642675280570984
0.3476184606552124
0.4270806610584259
0.2827416956424713
0.4032341241836548
0.5992004871368408
0.5999199748039246
0.314348429441452
0.49970436096191406
0.4096202850341797
0.8972026705741882
0.5311083197593689
0.3620019853115082
0.5005245208740234
0.5005245208740234
0.4683704376220703
0.34860172867774963
0.49017637968063354
0.3397650420665741
0.3192991316318512
0.3204971253871918
0.20938333868980408
0.2917713522911072
0.25660571455955505
0.25660571455955505
0.13604119420051575
0.13604119420051575
0.18702803552150726
0.024815574288368225
0.15993541479110718
0.3861231207847595
0.3192991316318512
0.3204971253871918
0.20938333868980408
0.2917713522911072
0.25660571455955505
0.25660571455955505
0.13604119420051575
0.13604119420051575
0.18702803552150726
0.024815574288368225
0.15993541479110718
0.3861231207847595
0.09852690994739532
0.3642675280570984
0.4270806610584259
0.16036072373390198
0.2827416956424713
0.2827416956424713
0.38009

0.372575968503952
0.287070095539093
0.39643317461013794
0.30370211601257324
0.33468011021614075
0.3178158700466156
0.3332929313182831
0.3576441705226898
0.20220337808132172
0.20220337808132172
0.16794827580451965
0.16794827580451965
0.23755063116550446
0.2085486799478531
0.1784890741109848
0.2866365909576416
0.27513161301612854
0.5239914059638977
0.7234411835670471
0.17016199231147766
0.5005245208740234
0.5005245208740234
0.2473173588514328
0.13119682669639587
0.4240686595439911
0.1919444352388382
0.497881144285202
0.7631610035896301
0.5564721822738647
0.45322221517562866
0.807807207107544
0.807807207107544
0.48122864961624146
0.48122864961624146
0.4797055423259735
0.315017431974411
0.4953540563583374
0.3430282771587372
0.45223692059516907
0.5765108466148376
0.4439809024333954
0.5754390954971313
0.503911554813385
0.503911554813385
0.281848669052124
0.281848669052124
0.36327770352363586
0.2671548128128052
0.3661086857318878
0.4611251652240753
0.45044586062431335
0.6231473088264465
0.457

In [50]:
sent_kp_df

,arg_id,topic,argument,num_of_aspects,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA,score,predicted_by_cosine
0,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_6,Over priced and overrated .,priced,over,over priced,negative,value-for-money,0.550575316,0.093817,True
1,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_8,Overall just poor quality .,quality,poor,poor quality,negative,restaurant -> atmosphere,0.530240297,0.215494,True
2,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_1,TERRIBLE CUSTOMER SERVICE !,customer service,terrible,terrible customer service,negative,staff,0.626694322,0.364268,True
3,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_5,Worst experience ever buying cars,experience,worst,worst experience,negative,restaurant -> atmosphere,0.471266836,0.347618,True
4,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_0,The cashiers are always friendly .,cashiers,always friendly,always friendly cashiers,positive,staff,0.429570019,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_13,Very courteous staff .,staff,very courteous,very courteous staff,positive,staff,0.427563697,0.000000,False
1022,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_14,Excellent selection of vehicles,vehicles,excellent selection of,excellent selection of vehicles,positive,food -> variety,0.458006501,0.000000,False
1023,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_15,"Free wifi , water , coffee , snacks !",wifi water coffee,free,free wifi water coffee,positive,wait-time,0.539548993,0.000000,False
1024,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_16,Their work is Impeccable .,work,impeccable,impeccable work,positive,staff,0.446787089,0.000000,False


In [51]:
import nltk
def label_implicit_matching_pair(row):
    token1 = row['aspects_x'][0]
    token2 = row['aspects_y'][0]
    
    # Best
    if row['aspects_x_len'] >= row['aspects_y_len']:
        if token2 in token1 and len(token1) > len(token2) and row['aspects_x_len'] > row['aspects_y_len']: # e.g. mexican food vs food
            row['label'] = 1
    return row

In [52]:
# matched_sent_kp_df[matched_sent_kp_df['aspects_similarity'] > 0.55] = matched_sent_kp_df[matched_sent_kp_df['aspects_similarity'] > 0.55].apply(label_implicit_matching_pair, axis=1)

In [53]:
# mask = (sent_kp_df['predicted_by_cosine'] == True)
# # sent_kp_df.loc[mask & (sent_kp_df['score'] > 0.55), 'score'] = 1
# sent_kp_df.loc[mask & (sent_kp_df['score'] <= 0.55), 'score'] = 0

In [54]:
sent_kp_df

,arg_id,topic,argument,num_of_aspects,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA,score,predicted_by_cosine
0,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_6,Over priced and overrated .,priced,over,over priced,negative,value-for-money,0.550575316,0.093817,True
1,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_8,Overall just poor quality .,quality,poor,poor quality,negative,restaurant -> atmosphere,0.530240297,0.215494,True
2,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_1,TERRIBLE CUSTOMER SERVICE !,customer service,terrible,terrible customer service,negative,staff,0.626694322,0.364268,True
3,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_5,Worst experience ever buying cars,experience,worst,worst experience,negative,restaurant -> atmosphere,0.471266836,0.347618,True
4,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_0,The cashiers are always friendly .,cashiers,always friendly,always friendly cashiers,positive,staff,0.429570019,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_13,Very courteous staff .,staff,very courteous,very courteous staff,positive,staff,0.427563697,0.000000,False
1022,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_14,Excellent selection of vehicles,vehicles,excellent selection of,excellent selection of vehicles,positive,food -> variety,0.458006501,0.000000,False
1023,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_15,"Free wifi , water , coffee , snacks !",wifi water coffee,free,free wifi water coffee,positive,wait-time,0.539548993,0.000000,False
1024,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_16,Their work is Impeccable .,work,impeccable,impeccable work,positive,staff,0.446787089,0.000000,False


In [55]:
# import nltk
# def label_implicit_matching_pair(row):
#     token1 = row['aspects_x'][0]
#     token2 = row['aspects_y'][0]
    
#     # Best
#     if row['aspects_x_len'] >= row['aspects_y_len']:
#         if token2 in token1 and len(token1) > len(token2) and row['aspects_x_len'] > row['aspects_y_len']: # e.g. mexican food vs food
#             row['label'] = 1

# #     if row['aspects_x_len'] >=     w['aspects_y_len']:
# #         if (token2 in token1) and len(token1) > len(token2): # e.g. mexican food vs food, prices vs price
# #             row['label'] = 1
# #             if row['aspects_x_len'] > row['aspects_y_len']:
# #                 display(row)

# #     elif row['aspects_x_len'] == row['aspects_y_len']:
# #         if nltk.edit_distance(token1, token2) <= 2 and len(token1) >= len(token2): # e.g. prices, price
# #             display(row)
# #             row['label'] = 1


# #     if row['aspects_x_len'] >= row['aspects_y_len']:
# #         if (token2 in token1 or nltk.edit_distance(token1, token2) <= 2) and\
# #             len(token1) >= len(token2): # e.g. mexican food vs food
# #             row['label'] = 1


            
# #     if token2 in token1 and len(token1) > len(token2) and row['aspects_x_len'] > row['aspects_y_len']: # e.g. mexican food vs food
# #         row['label'] = 2
# #     elif row['aspects_x_len'] == row['aspects_y_len']:
# #         row['label'] = 2
    
#     return row

In [56]:
# matched_sent_kp_df[matched_sent_kp_df['aspects_similarity'] > 0.55] = matched_sent_kp_df[matched_sent_kp_df['aspects_similarity'] > 0.55].apply(label_implicit_matching_pair, axis=1)

In [57]:
sent_kp_df

,arg_id,topic,argument,num_of_aspects,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA,score,predicted_by_cosine
0,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_6,Over priced and overrated .,priced,over,over priced,negative,value-for-money,0.550575316,0.093817,True
1,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_8,Overall just poor quality .,quality,poor,poor quality,negative,restaurant -> atmosphere,0.530240297,0.215494,True
2,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_1,TERRIBLE CUSTOMER SERVICE !,customer service,terrible,terrible customer service,negative,staff,0.626694322,0.364268,True
3,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_5,Worst experience ever buying cars,experience,worst,worst experience,negative,restaurant -> atmosphere,0.471266836,0.347618,True
4,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_0,The cashiers are always friendly .,cashiers,always friendly,always friendly cashiers,positive,staff,0.429570019,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_13,Very courteous staff .,staff,very courteous,very courteous staff,positive,staff,0.427563697,0.000000,False
1022,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_14,Excellent selection of vehicles,vehicles,excellent selection of,excellent selection of vehicles,positive,food -> variety,0.458006501,0.000000,False
1023,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_15,"Free wifi , water , coffee , snacks !",wifi water coffee,free,free wifi water coffee,positive,wait-time,0.539548993,0.000000,False
1024,arg_1_249_1,Automotive,Everybody is so rude & disrespectful !,1,everybody,rude,rude everybody,negative,staff,kp_1_16,Their work is Impeccable .,work,impeccable,impeccable work,positive,staff,0.446787089,0.000000,False


In [58]:
def select_best_kp_per_opinion(grp):
    best_match = grp.sort_values(by=['score'], ascending=False)
    best_match = best_match.iloc[0]
    return best_match
    
new_sent_kp_df = sent_kp_df.groupby(['topic', 'arg_id', 'opinion_aspect_pairs_x']).apply(select_best_kp_per_opinion).reset_index(drop=True)

In [59]:
new_sent_kp_df = new_sent_kp_df.drop_duplicates(subset=['arg_id', 'key_point_id'])
new_sent_kp_df

,arg_id,topic,argument,num_of_aspects,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,key_point,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA,score,predicted_by_cosine
0,arg_1_1001_1,Automotive,"Overall , Andrew provides a truly consultative process without any sales pressure .",1,process,truly consultative,truly consultative process,positive,staff,kp_1_2,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227013,0.547548,True
1,arg_1_1069_1,Automotive,"Rick , the owner , is very knowledgable and professional .",1,owner,very knowledgable,very knowledgable owner,positive,staff,kp_1_17,Store was clean and organized .,store,clean,clean store,positive,drink -> quality,0.569186211,0.341143,True
2,arg_1_1096_1,Automotive,Service team is truly customer focused .,1,service team,truly customer focused,truly customer focused service team,positive,staff,kp_1_9,FAST and FRIENDLY SERVICE ! ! !,service,fast and friendly,fast and friendly service,positive,wait-time,0.561472356,0.739000,True
3,arg_1_1102_1,Automotive,She is a very kind and respectful sales person .,1,sales person,very kind and respectful,very kind and respectful sales person,positive,staff,kp_1_2,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227013,0.676720,True
4,arg_1_1107_1,Automotive,Shop is super clean and well organized .,1,shop,super,super shop,positive,staff,kp_1_17,Store was clean and organized .,store,clean,clean store,positive,drink -> quality,0.569186211,0.787622,True
5,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_3,"employees looked overworked , confused .",employees,overworked,overworked employees,negative,staff,0.506309807,0.427081,True
6,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,quality,zero,zero quality,negative,restaurant -> atmosphere,kp_1_8,Overall just poor quality .,quality,poor,poor quality,negative,restaurant -> atmosphere,0.530240297,1.000000,False
7,arg_1_1153_1,Automotive,"Terry Repo is a consummate salesman , very honest and approachable .",1,salesman,approachable,approachable salesman,positive,staff,kp_1_17,Store was clean and organized .,store,clean,clean store,positive,drink -> quality,0.569186211,0.386123,True
8,arg_1_1222_1,Automotive,The cashier Amber was extremely friendly and efficient .,1,cashier,extremely friendly and efficient,extremely friendly and efficient cashier,positive,staff,kp_1_7,Prices were fair and reasonable .,prices,fair and reasonable,fair and reasonable prices,positive,value-for-money,0.499178082,0.357644,True
9,arg_1_1259_1,Automotive,The entire team is very caring and supportive .,1,team,very caring,very caring team,positive,staff,kp_1_13,Very courteous staff .,staff,very courteous,very courteous staff,positive,staff,0.427563697,0.531893,True


In [60]:
merged_df = pd.merge(new_sent_kp_df, labels_df, how="left", on=["arg_id", "key_point_id"])

In [61]:
# #merge each argument with the gold labels
# merged_df = pd.merge(sent_kp_df, labels_df, how="left", on=["arg_id", "key_point_id"])

In [62]:
merged_df.loc[merged_df['key_point_id'] == "dummy_id", 'label'] = 0
merged_df["label_strict"] = merged_df["label"].fillna(0)
merged_df["label_relaxed"] = merged_df["label"].fillna(1)

In [63]:
merged_df

,arg_id,topic,argument,num_of_aspects,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,...,opinions_y,opinion_aspect_pairs_y,sentiments_y,attributes_y,predicted_WA,score,predicted_by_cosine,label,label_strict,label_relaxed
0,arg_1_1001_1,Automotive,"Overall , Andrew provides a truly consultative process without any sales pressure .",1,process,truly consultative,truly consultative process,positive,staff,kp_1_2,...,excellent,excellent customer service,positive,staff,0.585227013,0.547548,True,1,1,1
1,arg_1_1069_1,Automotive,"Rick , the owner , is very knowledgable and professional .",1,owner,very knowledgable,very knowledgable owner,positive,staff,kp_1_17,...,clean,clean store,positive,drink -> quality,0.569186211,0.341143,True,0,0,0
2,arg_1_1096_1,Automotive,Service team is truly customer focused .,1,service team,truly customer focused,truly customer focused service team,positive,staff,kp_1_9,...,fast and friendly,fast and friendly service,positive,wait-time,0.561472356,0.739000,True,0,0,0
3,arg_1_1102_1,Automotive,She is a very kind and respectful sales person .,1,sales person,very kind and respectful,very kind and respectful sales person,positive,staff,kp_1_2,...,excellent,excellent customer service,positive,staff,0.585227013,0.676720,True,0,0,0
4,arg_1_1107_1,Automotive,Shop is super clean and well organized .,1,shop,super,super shop,positive,staff,kp_1_17,...,clean,clean store,positive,drink -> quality,0.569186211,0.787622,True,1,1,1
5,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,job,terrible,terrible job,negative,staff,kp_1_3,...,overworked,overworked employees,negative,staff,0.506309807,0.427081,True,0,0,0
6,arg_1_1148_2,Automotive,"Terrible job , overpriced for zero quality .",2,quality,zero,zero quality,negative,restaurant -> atmosphere,kp_1_8,...,poor,poor quality,negative,restaurant -> atmosphere,0.530240297,1.000000,False,1,1,1
7,arg_1_1153_1,Automotive,"Terry Repo is a consummate salesman , very honest and approachable .",1,salesman,approachable,approachable salesman,positive,staff,kp_1_17,...,clean,clean store,positive,drink -> quality,0.569186211,0.386123,True,0,0,0
8,arg_1_1222_1,Automotive,The cashier Amber was extremely friendly and efficient .,1,cashier,extremely friendly and efficient,extremely friendly and efficient cashier,positive,staff,kp_1_7,...,fair and reasonable,fair and reasonable prices,positive,value-for-money,0.499178082,0.357644,True,0,0,0
9,arg_1_1259_1,Automotive,The entire team is very caring and supportive .,1,team,very caring,very caring team,positive,staff,kp_1_13,...,very courteous,very courteous staff,positive,staff,0.427563697,0.531893,True,1,1,1


In [64]:
def get_ap(df, label_column, top_percentile=0.5):
    top = int(len(df)*top_percentile)
    df = df.sort_values('score', ascending=False)
    if top != 0:
        df = df.head(top)
    # after selecting top percentile candidates, we set the score for the dummy kp to 1, to prevent it from increasing the precision.
    df.loc[df['key_point_id'] == "dummy_id", 'score'] = 0.99
#     print("NEW")
#     display(df)
#     display(df[pd.isnull(df)])
#     print()
    return average_precision_score(y_true=df[label_column], y_score=df["score"])

def calc_mean_average_precision(df, label_column):
    precisions = [get_ap(group, label_column) for _, group in df.groupby(["topic"])]
#     precisions = [0 if pd.isnull(pre) else pre for pre in precisions]
#     precisions = [pre for pre in precisions if pd.notnull(pre)]
    print(precisions)
    return np.mean(precisions)

def evaluate_predictions(merged_df):
    mAP_strict = calc_mean_average_precision(merged_df, "label_strict")
    mAP_relaxed = calc_mean_average_precision(merged_df, "label_relaxed")
    return mAP_strict, mAP_relaxed

In [65]:
[(t, get_ap(group, "label_strict")) for t, group in merged_df.groupby(["topic"])]

[('Automotive', 0.5822776003554035)]

In [66]:
[(t, get_ap(group, "label_relaxed")) for t, group in merged_df.groupby(["topic"])]

[('Restaurants', 0.7765644827355906)]

In [63]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.8962692164780265, 0.8068616317097077]
[0.8962692164780265, 0.8068616317097077]
mAP strict= 0.851565424093867 ; mAP relaxed = 0.851565424093867


### Hotels

Multi-Aspect

In [64]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.879320987654321]
[0.879320987654321]
mAP strict= 0.879320987654321 ; mAP relaxed = 0.879320987654321


Combined

In [65]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.9538051289328429]
[0.9538051289328429]
mAP strict= 0.9538051289328429 ; mAP relaxed = 0.9538051289328429


### Restaurants

Multi-Aspect

In [69]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.716017316017316]
[0.716017316017316]
mAP strict= 0.716017316017316 ; mAP relaxed = 0.716017316017316


Combined

In [63]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.7765644827355906]
[0.7765644827355906]
mAP strict= 0.7765644827355906 ; mAP relaxed = 0.7765644827355906


### Arts & Entertainment

Multi-Aspect

In [66]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.92]
[0.92]
mAP strict= 0.92 ; mAP relaxed = 0.92


Combined

In [66]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.9230769230769231]
[0.9230769230769231]
mAP strict= 0.9230769230769231 ; mAP relaxed = 0.9230769230769231


### Automotive

Multi-Aspect

In [69]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.42857142857142855]
[0.42857142857142855]
mAP strict= 0.42857142857142855 ; mAP relaxed = 0.42857142857142855


Combined

In [63]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.5822776003554035]
[0.5822776003554035]
mAP strict= 0.5822776003554035 ; mAP relaxed = 0.5822776003554035


### Beauty & Spas

Multi-Aspect

In [69]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.823639455782313]
[0.823639455782313]
mAP strict= 0.823639455782313 ; mAP relaxed = 0.823639455782313


Combined

In [63]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.8496957587866678]
[0.8496957587866678]
mAP strict= 0.8496957587866678 ; mAP relaxed = 0.8496957587866678


Multi-aspect

In [55]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.8844444444444444, 0.7924607706370156]
[0.8844444444444444, 0.7924607706370156]
mAP strict= 0.8384526075407299 ; mAP relaxed = 0.8384526075407299


In [ ]:
# Topic merge only, with no best-matching
# Only 0.42

In [317]:
# Topic merge only, with best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.8962692164780265, 0.8068616317097077]
[0.8962692164780265, 0.8068616317097077]
mAP strict= 0.851565424093867 ; mAP relaxed = 0.851565424093867


In [247]:
# Topic vs aspect category merge, but no best-matching
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.7078265760989924, 0.6041469582888587]
[0.7078265760989924, 0.6041469582888587]
mAP strict= 0.6559867671939255 ; mAP relaxed = 0.6559867671939255


In [262]:
# 30%
# Topic vs aspect category merge, with best-matching
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.9216653367978361, 0.8017873848903975]
[0.9216653367978361, 0.8017873848903975]
mAP strict= 0.8617263608441168 ; mAP relaxed = 0.8617263608441168


With 0.55 Rounding Neg Only

In [366]:
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.5524090137049532, 0.40549477577266085]
[0.5524090137049532, 0.40549477577266085]
mAP strict= 0.47895189473880706 ; mAP relaxed = 0.47895189473880706


In [381]:
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.9314898262807844, 0.8012792037619579]
[0.9314898262807844, 0.8012792037619579]
mAP strict= 0.8663845150213711 ; mAP relaxed = 0.8663845150213711


With 0.55 Rounding Pos, Neg

In [390]:
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.2771587151629665, 0.19557911566790592]
[0.2771587151629665, 0.19557911566790592]
mAP strict= 0.2363689154154362 ; mAP relaxed = 0.2363689154154362


In [396]:
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.41304347826086957, 0.3333333333333333]
[0.41304347826086957, 0.3333333333333333]
mAP strict= 0.37318840579710144 ; mAP relaxed = 0.37318840579710144


In [322]:
# 30%
#Perform evaluation
mAP_strict, mAP_relaxed = evaluate_predictions(merged_df)

print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

# logger.info("mAP strict:   \t{:.2f}".format(mAP_strict*100))
# logger.info("mAP relaxed:   \t{:.2f}".format(mAP_relaxed*100))

[0.4690003763949305, 0.3862436420715855]
[0.4690003763949305, 0.3862436420715855]
mAP strict= 0.427622009233258 ; mAP relaxed = 0.427622009233258


### Aspect-based Matching KP Coverage Test 2

In [73]:
valid_df = df[(df['label'] == 1) & (df['arg_id'].str.startswith('arg_3'))]
valid_kp = valid_df['full_key_point'].value_counts()
valid_kp

Friendly and helpful staff .              21
Clean and comfortable rooms .             13
Excellent customer service .               7
The service is impeccable .                5
Very peaceful and relaxing location .      4
Great desk service , too .                 4
Extremely poor service .                   3
The beds are extremely comfortable .       2
Pool area clean and refreshing .           2
Price way overvalued .                     1
The ambience is wonderfully peaceful .     1
Horribly rude staff .                      1
Front desk staff was inept .               1
Room decor is outdated .                   1
Wifi was very reliable .                   1
Valet is convenient and fast .             1
Parking is easy .                          1
Restaurant service absolutely sucks .      1
Beds were extremely uncomfortable .        1
Great cocktail list too .                  1
Name: full_key_point, dtype: int64

In [74]:
# valid_kp[valid_kp > 3]

In [75]:
# merged_df[merged_df['arg_id'].str.startswith('arg_3')].merge(kp_df, on=['key_point_id'])['full_key_point'].tolist()

In [76]:
merged_df.columns

Index(['arg_id', 'topic', 'argument', 'num_of_aspects', 'aspects_x',
       'opinions_x', 'opinion_aspect_pairs_x', 'sentiments_x', 'attributes_x',
       'key_point_id', 'key_point', 'aspects_y', 'opinions_y',
       'opinion_aspect_pairs_y', 'sentiments_y', 'attributes_y',
       'predicted_WA', 'score', 'predicted_by_cosine', 'label', 'label_strict',
       'label_relaxed'],
      dtype='object')

In [77]:
# Hotel Only
top_kp = merged_df[merged_df['arg_id'].str.startswith('arg_3')].merge(kp_df, on=['key_point_id'])['key_point_x'].value_counts()
result = top_kp[top_kp.index.isin(valid_kp.index.tolist())]
result

Friendly and helpful staff .              16
Clean and comfortable rooms .             12
Very peaceful and relaxing location .      3
Restaurant service absolutely sucks .      3
The service is impeccable .                2
Pool area clean and refreshing .           2
The ambience is wonderfully peaceful .     2
Great desk service , too .                 2
Room decor is outdated .                   2
Horribly rude staff .                      1
The beds are extremely comfortable .       1
Excellent customer service .               1
Front desk staff was inept .               1
Parking is easy .                          1
Wifi was very reliable .                   1
Name: key_point_x, dtype: int64

In [72]:
selected_kps = [
    'Friendly and courteous staff .  ',
    'Clean and comfortable rooms .',
    'Great food selection',
    'Poor service .',
#     'Beds were comfortable .',
    'The food is less consistent .  ',
#     'Not worth the money'
]

In [83]:
# valid_df = valid_df[valid_df['full_key_point'].isin(selected_kps)]

In [84]:
valid_df = valid_df[valid_df['full_key_point'].isin(valid_kp.head(2).index)]

In [85]:
valid_df['full_key_point'].value_counts()

They have excellent customer service .    29
Very professional staff .                 13
Name: full_key_point, dtype: int64

In [86]:
valid_df.shape

(42, 16)

In [89]:
hotel_pred_df = merged_df[merged_df['arg_id'].str.startswith('arg_1')].merge(kp_df, on=['key_point_id'])
# hotel_pred_df = hotel_pred_df[hotel_pred_df['full_key_point'].isin(selected_kps)]
hotel_pred_df = hotel_pred_df[hotel_pred_df['key_point_x'].isin(valid_kp.head(2).index)]
hotel_pred_df['pred_label'] = 1
hotel_pred_df

,arg_id,topic_x,argument,num_of_aspects,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,attributes_x,key_point_id,...,label_relaxed,topic_y,key_point_y,aspects,opinions,opinion_aspect_pairs,sentiments,attributes,predicted_WA_y,pred_label
0,arg_1_1001_1,Automotive,"Overall , Andrew provides a truly consultative process without any sales pressure .",1,process,truly consultative,truly consultative process,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
1,arg_1_1102_1,Automotive,She is a very kind and respectful sales person .,1,sales person,very kind and respectful,very kind and respectful sales person,positive,staff,kp_1_2,...,0,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
2,arg_1_1454_2,Automotive,Their dedication to resolving this major safety problem was incredible .,2,safety problem,incredible,incredible safety problem,positive,restaurant -> atmosphere,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
3,arg_1_1520_2,Automotive,They have great offers and awesome customer service too .,2,customer service,awesome,awesome customer service,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
4,arg_1_1525_1,Automotive,"They have such amazing customer service , it was n't just them",1,customer service,amazing,amazing customer service,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
5,arg_1_1553_1,Automotive,"They were fast , friendly , & the customer service was superb !",1,customer service,superb,superb customer service,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
6,arg_1_1652_1,Automotive,Tracie Seib gave us outstanding customer service .,1,customer service,outstanding,outstanding customer service,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
7,arg_1_1661_1,Automotive,Use the check - in offer on Yelp for a solid discount .,1,discount,solid,solid discount,positive,value-for-money,kp_1_2,...,0,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
8,arg_1_361_1,Automotive,Great communication between staff and customers .,1,communication,great,great communication,positive,restaurant -> atmosphere,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1
9,arg_1_95_1,Automotive,Audi has the best customer service & streamlined purchase program .,1,customer service,best,best customer service,positive,staff,kp_1_2,...,1,Automotive,They have excellent customer service .,customer service,excellent,excellent customer service,positive,staff,0.585227,1


In [90]:
hotel_pred_df.shape

(10, 31)

In [91]:
# valid_df = valid_df.merge(hotel_pred_df[['arg_id', 'key_point_id', 'pred_label']], how='left')
# valid_df['pred_label'] = valid_df['pred_label'].fillna(0)
# valid_df

In [92]:
from sklearn.metrics import recall_score, precision_score

new_valid_df = valid_df.merge(hotel_pred_df[['arg_id', 'key_point_id', 'pred_label']], how='right')
new_valid_df['label'] = new_valid_df['label'].fillna(0)
print(precision_score(new_valid_df['label'], new_valid_df['pred_label']))

new_valid_df = valid_df.merge(hotel_pred_df[['arg_id', 'key_point_id', 'pred_label']], how='left')
new_valid_df['pred_label'] = new_valid_df['pred_label'].fillna(0)
print(recall_score(new_valid_df['label'], new_valid_df['pred_label']))

0.8
0.19047619047619047


In [66]:
domain_mapping = {
    'Arts & Entertainment': 0,
    'Automotive': 1,
    'Beauty & Spas': 2,
    'Hotels': 3,
    'Restaurants': 4
}
from sklearn.metrics import recall_score, precision_score

def calculate_precision_recall(merged_df, df, domain, top=None, selected_kps=[]):
    valid_df = df[(df['label'] == 1) & (df['arg_id'].str.startswith(f'arg_{domain_mapping[domain]}'))]
    valid_kp = valid_df['full_key_point'].value_counts()
    
    top_kp = merged_df.drop(['key_point'], axis=1)[merged_df['arg_id'].str.startswith(f'arg_{domain_mapping[domain]}')]\
        .merge(kp_df, on=['key_point_id'])['key_point'].value_counts()
    result = top_kp[top_kp.index.isin(valid_kp.index.tolist())]
    if top != None:
        display(result[result.index.isin(valid_kp.head(top).index)])
    elif len(selected_kps) > 0:
        display(result[result.index.isin(selected_kps)])
    else:
        display(result)
    
    if top != None:
        valid_df = valid_df[valid_df['full_key_point'].isin(valid_kp.head(top).index)]
    elif len(selected_kps) > 0:
        valid_df = valid_df[valid_df['full_key_point'].isin(selected_kps)]
    
    pred_df = merged_df[merged_df['arg_id'].str.startswith(f'arg_{domain_mapping[domain]}')].drop(['key_point'], axis=1)\
        .merge(kp_df, on=['key_point_id'])
    if top != None:
        pred_df = pred_df[pred_df['key_point'].isin(valid_kp.head(top).index)]
    elif len(selected_kps) > 0:
        pred_df = pred_df[pred_df['key_point'].isin(selected_kps)]
    
#     pred_df = pred_df[pred_df['score'] >= 0.5]
#     pred_df = pred_df[pred_df['score'] >= 0.2]
    pred_df['pred_label'] = 1
#     pred_df.loc[pred_df['score'] >= 0.5 ,'pred_label'] = 1
#     pred_df.loc[pred_df['score'] < 0.5 ,'pred_label'] = 0
    
    new_valid_df = valid_df.merge(pred_df[['arg_id', 'key_point_id', 'pred_label']], how='right')
    new_valid_df['label'] = new_valid_df['label'].fillna(0)
    precision = precision_score(new_valid_df['label'], new_valid_df['pred_label'])

    new_valid_df = valid_df.merge(pred_df[['arg_id', 'key_point_id', 'pred_label']], how='left')
    new_valid_df['pred_label'] = new_valid_df['pred_label'].fillna(0)
    recall = recall_score(new_valid_df['label'], new_valid_df['pred_label'])
    
    return precision, recall

In [151]:
# merged_df = merged_df.drop_duplicates(subset=['arg_id'])

In [113]:
# merged_df.set_index("arg_id")["key_point_id"].to_dict()

In [67]:
calculate_precision_recall(merged_df, df, 'Arts & Entertainment', 2)

Friendly and helpful staff .       10
Awesome service and employees .     3
Name: key_point, dtype: int64

(0.9230769230769231, 0.5)

In [68]:
calculate_precision_recall(merged_df, df, 'Arts & Entertainment', 3)

Friendly and helpful staff .       10
Awesome service and employees .     3
It had wonderful acoustics .        1
Name: key_point, dtype: int64

(0.9285714285714286, 0.3939393939393939)

In [69]:
calculate_precision_recall(merged_df, df, 'Arts & Entertainment', None)

Friendly and helpful staff .                10
Superior customer service !                  5
Very talented musicians !                    4
Seats are adequately comfortable .           4
Parking was fast and FREE !                  4
Horrible customer service .                  3
Rude and Unprofessional staff .              3
Awesome service and employees .              3
Incredibly talented performers - wow !       2
Nice employees ... not to pushy .            2
Awesome lighting , and decor .               1
It had wonderful acoustics .                 1
Worth the money .                            1
lines were short and quick .                 1
Terrible administration and management .     1
The acoustics are poor .                     1
Name: key_point, dtype: int64

(0.5172413793103449, 0.43478260869565216)

In [89]:
selected_kps = ['Friendly and helpful staff .', 'Seats are adequately comfortable .   ', 'Horrible customer service .']
calculate_precision_recall(merged_df, df, 'Arts & Entertainment', None, selected_kps)

Friendly and helpful staff .             10
Seats are adequately comfortable .        4
Horrible customer service .               3
Name: key_point, dtype: int64

(0.9411764705882353, 0.7619047619047619)

In [68]:
calculate_precision_recall(merged_df, df, 'Automotive', 2)

They have excellent customer service .    10
Name: key_point, dtype: int64

(0.8, 0.19047619047619047)

In [69]:
calculate_precision_recall(merged_df, df, 'Automotive', 3)

They have excellent customer service .    10
The employees here are wonderful !         2
Name: key_point, dtype: int64

(0.8333333333333334, 0.18181818181818182)

In [70]:
calculate_precision_recall(merged_df, df, 'Automotive', None)

FAST and FRIENDLY SERVICE ! ! !           11
They have excellent customer service .    10
Very courteous staff .                     9
Store was clean and organized .            6
Their work is Impeccable .                 5
The employees here are wonderful !         2
The cashiers are always friendly .         2
Overall just poor quality .                1
Prices were fair and reasonable .          1
TERRIBLE CUSTOMER SERVICE !                1
Extremely poor service .                   1
Name: key_point, dtype: int64

(0.4, 0.23655913978494625)

In [75]:
# selected_kps = ['FAST and FRIENDLY SERVICE ! ! !', 'They have excellent customer service .', 'Store was clean and organized .  ']
# calculate_precision_recall(merged_df, df, 'Automotive', None, selected_kps)

In [76]:
# selected_kps = ['FAST and FRIENDLY SERVICE ! ! !', 'The employees here are wonderful !', 'Store was clean and organized .  ']
# calculate_precision_recall(merged_df, df, 'Automotive', None, selected_kps)

In [77]:
# selected_kps = ['Store was clean and organized .  ', 'The employees here are wonderful !', 'Very professional staff .']
# calculate_precision_recall(merged_df, df, 'Automotive', None, selected_kps)

In [67]:
calculate_precision_recall(merged_df, df, 'Beauty & Spas', 2)

Staff is friendly and accommodating .    13
Amazing & professional service .          3
Name: key_point, dtype: int64

(0.9375, 0.46875)

In [68]:
calculate_precision_recall(merged_df, df, 'Beauty & Spas', 3)

Staff is friendly and accommodating .    13
Customer service- EXCELLENT !             7
Amazing & professional service .          3
Name: key_point, dtype: int64

(0.8695652173913043, 0.4444444444444444)

In [69]:
calculate_precision_recall(merged_df, df, 'Beauty & Spas', None)

Staff is friendly and accommodating .    13
Customer service- EXCELLENT !             7
Super clean and friendly salon .          5
Very professionally run business .        5
Amazing & professional service .          3
Artists are highly skilled .              3
This place is TERRIFIC .                  3
Super comfy and great massage .           2
POOR CUSTOMER SERVICE .                   2
Prices were fair and reasonable .         1
Name: key_point, dtype: int64

(0.5357142857142857, 0.38961038961038963)

In [108]:
selected_kps = ['Staff is friendly and accommodating .', 'Customer service- EXCELLENT !',  'This place is TERRIFIC .  ']
calculate_precision_recall(merged_df, df, 'Beauty & Spas', None, selected_kps)

Staff is friendly and accommodating .    13
Customer service- EXCELLENT !             7
This place is TERRIFIC .                  3
Name: key_point, dtype: int64

(0.8260869565217391, 0.59375)

In [67]:
calculate_precision_recall(merged_df, df, 'Hotels', 2)

Series([], Name: key_point, dtype: int64)

UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

In [68]:
calculate_precision_recall(merged_df, df, 'Hotels', 3)

Friendly and helpful staff .     16
Clean and comfortable rooms .    12
Excellent customer service .      1
Name: key_point, dtype: int64

(0.9655172413793104, 0.6829268292682927)

In [69]:
calculate_precision_recall(merged_df, df, 'Hotels', None)

Friendly and helpful staff .              16
Clean and comfortable rooms .             12
Very peaceful and relaxing location .      3
Restaurant service absolutely sucks .      3
The service is impeccable .                2
Pool area clean and refreshing .           2
The ambience is wonderfully peaceful .     2
Great desk service , too .                 2
Room decor is outdated .                   2
Horribly rude staff .                      1
The beds are extremely comfortable .       1
Excellent customer service .               1
Front desk staff was inept .               1
Parking is easy .                          1
Wifi was very reliable .                   1
Name: key_point, dtype: int64

(0.6724137931034483, 0.5416666666666666)

In [70]:
# sorted_kp_id = valid_df['key_point_id'].value_counts().index
# # print(sorted_kp_id)
# # new_df = df[df['isMultiAspect'] == False]
# new_df = df
# new_df = new_df.set_index(['key_point_id']).loc[sorted_kp_id].reset_index().rename(columns={'index': 'key_point_id'})
# # new_df = new_df.drop_duplicates(subset='arg_id')
# new_df = new_df.groupby(['arg_id']).apply(lambda grp: grp.head(len(grp['attributes'].iloc[0]))).reset_index(drop=True)
# new_df

In [71]:
selected_kps = ['Friendly and helpful staff .', 'Clean and comfortable rooms .', 
                'The service is impeccable .']
calculate_precision_recall(merged_df, df, 'Hotels', None, selected_kps)

Friendly and helpful staff .     16
Clean and comfortable rooms .    12
The service is impeccable .       2
Name: key_point, dtype: int64

(0.9333333333333333, 0.717948717948718)

In [72]:
selected_kps = ['Friendly and helpful staff .', 'Clean and comfortable rooms .', 'The ambience is wonderfully peaceful .']
calculate_precision_recall(merged_df, df, 'Hotels', None, selected_kps)

Friendly and helpful staff .              16
Clean and comfortable rooms .             12
The ambience is wonderfully peaceful .     2
Name: key_point, dtype: int64

(0.9333333333333333, 0.8)

In [68]:
calculate_precision_recall(merged_df, df, 'Restaurants', 2)

Staff was courteous and accommodating .    11
Customer Service is Excellent .             7
Name: key_point, dtype: int64

(0.9444444444444444, 0.5151515151515151)

In [69]:
calculate_precision_recall(merged_df, df, 'Restaurants', 3)

Staff was courteous and accommodating .    11
Customer Service is Excellent .             7
Name: key_point, dtype: int64

(0.9444444444444444, 0.3617021276595745)

In [70]:
calculate_precision_recall(merged_df, df, 'Restaurants', None)

Staff was courteous and accommodating .       11
Fresh food , using local produce .             8
Customer Service is Excellent .                7
The service here was exceptional .             5
Lots of outdoor seating and                    3
AMAZING AUTHENTIC FLAVOR !                     3
Way over priced .                              2
Fantastic drink selection .                    2
Great beer and wine selection .                2
The quality has diminished significantly .     2
Great variety of menu items .                  2
Generous sized portions .                      2
Prices are fair and reasonable .               1
Plenty of coffees and teas .                   1
Name: key_point, dtype: int64

(0.484375, 0.28703703703703703)

In [80]:
# selected_kps = ['Staff was courteous and accommodating .', 
#                 'Fresh food , using local produce .', 'The service here was exceptional .  ']
selected_kps = ['Staff was courteous and accommodating .', 
                'Fresh food , using local produce .', 'Generous sized portions .']
calculate_precision_recall(merged_df, df, 'Restaurants', None, selected_kps)

Staff was courteous and accommodating .    11
Fresh food , using local produce .          8
Generous sized portions .                   2
Name: key_point, dtype: int64

(0.8095238095238095, 0.6296296296296297)